In [103]:
# imports
import sys
sys.path.append('../') 
import requests
import pandas as pd
import os # use this to access your environment variables
from citybikes import get_city_bikes_info
from apicall import get_venues_fs,query_yelp
import json


# Foursquare

POI - Vegetarian, Library
| City Bike Location - Toronto

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
FOURSQUARE_KEY = os.environ['FOURSQUARE_KEY']
foursquare_api_key = "FOURSQUARE_KEY"
yelp_api_key = "YELP_KEY"
network_id="bixi-toronto"
df_four=pd.DataFrame()
df_yelp=pd.DataFrame()

#call citybikes to fecth location details for all sations in the selected city 
# I have ran this api call to fetch citybikes station , lat, log and saved it to a csv file. 
# fetching info for this from csv file

file_path = '/Users/priyaganesan/Desktop/python_exercise/Pandas_exercise/db/Final-Project-statistical-Modelling-with-Python/bike_info_csv.csv'

# Read the CSV file into a DataFrame
bike_info_df = pd.read_csv(file_path)

point_of_interest_coordinates = bike_info_df[['latitude', 'longitude','name']]
#point_of_interest_coordinates=point_of_interest_coordinates.head(5)
# for each station lat,long call foursquare 
for index, row in point_of_interest_coordinates.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_name=row['name']
    #Call function to fetch points of interest near each lat,long from foursquare ,12080-library
    #,13377-vegetarian friendly
    #CALL FOURSQUARE API
    f_res=get_venues_fs(latitude,longitude,1000,four_square_key,'13377,12080') 
    #copy foursquare json data into df 
    fdf=pd.DataFrame(["category","name","tel","tips","distance","latitude","longitude","hours","address","station_name"])
    extracted_data_four=[
        {
            'name': results.get('name'),
            'distance':results.get('distance'),
            'category': results.get('categories', [])[0].get('short_name'),
            'latitude': results.get('geocodes', {}).get('main', {}).get('latitude'),
            'longitude': results.get('geocodes', {}).get('main', {}).get('longitude'),
            'rating': results.get('rating'),
            'reviews':results.get('stats',{}).get('total_ratings'),
            'description':results.get('description'),
            'formatted_address': results.get('location', {}).get('formatted_address'),
            'telephone': results.get('tel'),
            'tips': results.get('tips', [])[0].get('text') if results.get('tips') else None,
            'hours': results.get('hours', {}).get('display'),
            'station_name': station_name
        }
        for results in f_res.get('results', [])
    ]
    #COPY PARSED FLATTENED DATA TO DATAFRAME
    df = pd.DataFrame(extracted_data_four)
    df_four=df_four._append(df,ignore_index=True)

    #CALL YELP API 
    y_res=query_yelp(latitude,longitude,yelp_api_key) 
    #print(y_res)
    #copy json yelp data into df 
    ydy=pd.DataFrame(["name","closed","review","rating","categories","phone","address","distance","latitude","longitude","station_name"])
    for business in y_res.get('businesses', []):
    # Extract display address as a string
        display_address = ', '.join(business.get('location', {}).get('display_address', []))
    
    extracted_data_yelp=[
        {
            'name': business.get('name'),
            'closed':business.get('is_closed'),
            'review':business.get('review_count'),
            'rating':business.get('rating'),
            'categories': business.get('categories', [])[0].get('title'),
            'phone':business.get('phone'),
            'distance':business.get('distance'),
            'latitude': business.get('coordinates', {}).get('latitude'),
            'longitude': business.get('coordinates', {}).get('longitude'),
            'address':display_address,
            #'address':business.get('location', {}).get('display_address'),
            'station_name': station_name
        }
        for business in y_res.get('businesses', [])
    ]
    #COPY PARSED FLATTENED DATA TO DATAFRAME
    df = pd.DataFrame(extracted_data_yelp)
    df_yelp=df_yelp._append(df,ignore_index=True)  
#copy df into csv for further analysis
df_four.to_csv('/Users/priyaganesan/Desktop/python_exercise/Pandas_exercise/db/Final-Project-statistical-Modelling-with-Python/foursquare.csv', index=False)  
df_yelp.to_csv('/Users/priyaganesan/Desktop/python_exercise/Pandas_exercise/db/Final-Project-statistical-Modelling-with-Python/yelp.csv', index=False)  

  



In [193]:
bike_info_df.head(10)

,0,name,free_bikes,latitude,longitude
0,NaN,Queen St E / Woodward Ave,7.0,43.665269,-79.319796
1,NaN,Primrose Ave / Davenport Rd,3.0,43.671420,-79.445947
2,NaN,Queen St E / Rhodes Ave,20.0,43.666224,-79.317693
3,NaN,Bond St / Queen St E,31.0,43.653236,-79.376716
4,NaN,Church St / Alexander St,14.0,43.663722,-79.380288
5,NaN,University Ave / Gerrard St W (East Side),18.0,43.657763,-79.389165
6,NaN,Edward St / Yonge St,23.0,43.656729,-79.382736
7,NaN,Euclid Ave / Bloor St W,0.0,43.664467,-79.414783
8,NaN,Yonge St / Aylmer Ave,11.0,43.675492,-79.388858
9,NaN,Davenport Rd / Avenue Rd,3.0,43.674991,-79.396273


In [195]:
df_yelp.head()

,name,closed,review,rating,categories,phone,distance,latitude,longitude,address,station_name
0,Udupi Palace,False,208,3.5,Indian,+14164058189,782.410965,43.672045,-79.322420,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
1,KaSpace Cafe,False,9,5.0,Vegetarian,+16476579600,1023.018862,43.662630,-79.331900,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
2,Bombay Chowpatty,False,59,3.0,Indian,+14164058080,832.357744,43.671982,-79.324374,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
3,The Social Gardener,False,5,3.5,Vegan,+14164656021,839.372147,43.671644,-79.325385,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
4,Gerrard/Ashdale Library,False,4,3.5,Libraries,+14163937717,802.887228,43.672482,-79.322082,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [219]:
df_four.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               4216 non-null   object 
 1   distance           4216 non-null   int64  
 2   category           4216 non-null   object 
 3   latitude           4216 non-null   float64
 4   longitude          4216 non-null   float64
 5   rating             2104 non-null   float64
 6   reviews            2104 non-null   float64
 7   description        583 non-null    object 
 8   formatted_address  4216 non-null   object 
 9   telephone          4102 non-null   object 
 10  tips               2812 non-null   object 
 11  hours              3649 non-null   object 
 12  station_name       4216 non-null   object 
dtypes: float64(4), int64(1), object(8)
memory usage: 428.3+ KB


In [198]:
df_yelp.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          3677 non-null   object 
 1   closed        3677 non-null   bool   
 2   review        3677 non-null   int64  
 3   rating        3677 non-null   float64
 4   categories    3677 non-null   object 
 5   phone         3677 non-null   object 
 6   distance      3677 non-null   float64
 7   latitude      3677 non-null   float64
 8   longitude     3677 non-null   float64
 9   address       3677 non-null   object 
 10  station_name  3677 non-null   object 
dtypes: bool(1), float64(4), int64(1), object(5)
memory usage: 291.0+ KB


In [199]:
df_yelp.head()

,name,closed,review,rating,categories,phone,distance,latitude,longitude,address,station_name
0,Udupi Palace,False,208,3.5,Indian,+14164058189,782.410965,43.672045,-79.322420,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
1,KaSpace Cafe,False,9,5.0,Vegetarian,+16476579600,1023.018862,43.662630,-79.331900,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
2,Bombay Chowpatty,False,59,3.0,Indian,+14164058080,832.357744,43.671982,-79.324374,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
3,The Social Gardener,False,5,3.5,Vegan,+14164656021,839.372147,43.671644,-79.325385,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
4,Gerrard/Ashdale Library,False,4,3.5,Libraries,+14163937717,802.887228,43.672482,-79.322082,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave


In [200]:
df_yelp.tail()

,name,closed,review,rating,categories,phone,distance,latitude,longitude,address,station_name
3672,Freshii,False,6,2.0,Salad,+16474848550,790.938858,43.651226,-79.364586,"100 Queen Street W, Nathan Phillips Square, To...",Richmond St E / Jarvis St Green P
3673,City Hall Public Library,False,6,3.5,Libraries,+14163937650,776.740866,43.653347,-79.383826,"100 Queen Street W, Nathan Phillips Square, To...",Richmond St E / Jarvis St Green P
3674,Toronto Public Library - Parliament Street Branch,False,6,2.0,Libraries,+14163937663,314.733258,43.661830,-79.367910,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St
3675,Toronto Public Library - Riverdale,False,4,4.0,Libraries,+14163937720,1010.279295,43.665810,-79.353140,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St
3676,Toronto Public Library - St. James Town Branch,False,3,4.0,Libraries,+14163937744,1068.656684,43.667890,-79.374690,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

FOURSQUARE

Get the top 10 restaurants according to their rating

Top 10 POI- Vegetarian restaurant by rating

Remove duplicates and consider POI NAME, RATING, REVIEW, ADDRESS, TELEPHONE TO DISPLAY TOP 10 RATED RESTAURANT

In [228]:
columns_to_copy=['name','category','rating','reviews','formatted_address','telephone']
df_four_selectedcolumn=df_four[columns_to_copy].copy()
df_four_selectedcolumn=df_four_selectedcolumn.drop_duplicates()


In [241]:
foursquare_top_poi=df_four_selectedcolumn.sort_values(by='rating',ascending=False)
foursquare_top_resturants=foursquare_top_poi[foursquare_top_poi['category']!='Library'].head(20)
foursquare_top_resturants


,name,category,rating,reviews,formatted_address,telephone
368,Fresh,Vegan and Vegetarian Restaurant,7.8,140.0,"90 Eglinton Ave E (Yonge & Eglinton), Toronto ...",(416) 599-4442
31,Saigon Lotus,Vegan and Vegetarian Restaurant,7.8,18.0,"6 St Andrew St (Spadina Ave), Toronto ON M5T 1K6",(416) 901-1115
347,Gia,Vegan and Vegetarian Restaurant,7.8,9.0,"1214 Dundas St W, Toronto ON M6J 1X5",(416) 535-8888
237,King's Cafe,Café,7.7,67.0,"192 Augusta Ave, Toronto ON M5T 2L6",(416) 591-1340
324,Grey Tiger,Cocktail,7.7,16.0,"1190 Bloor St W, Toronto ON M6H 1N2",(647) 925-3310


In [238]:
foursquare_top_libraries=foursquare_top_poi[foursquare_top_poi['category']=='Library'].head(20)
foursquare_top_libraries

,name,category,rating,reviews,formatted_address,telephone
59,Toronto Public Library - Toronto Reference Lib...,Library,9.0,378.0,"789 Yonge St (btwn Collier & Asquith Ave), Tor...",(416) 395-5577
189,Toronto Public Library,Library,8.6,42.0,"190 Fort York Blvd, Toronto ON M5V 0E7",(416) 393-6240
30,Toronto Public Library,Library,8.3,38.0,"239 College St (at Huron St.), Toronto ON M5T 1R5",(416) 393-7746
89,Toronto Public Library,Library,8.0,11.0,"370 Broadview Ave (at Gerrard St E), Toronto O...",(416) 393-7720
325,Toronto Public Library,Library,7.9,51.0,"1101 Bloor St W (at Gladstone), Toronto ON M6H...",(416) 393-7674
1597,Toronto Public Library,Library,7.8,15.0,"3083 Yonge St (at Lawrence Ave.), Toronto ON M...",(416) 393-7730
146,Toronto Public Library,Library,7.6,14.0,"701 Pape Ave (at Danforth Ave), Toronto ON M4K...",(416) 393-7727
3687,Toronto Public Library,Library,7.6,8.0,"496 Birchmount Rd, Toronto ON M1K 1N8",(416) 396-8890
443,Toronto Public Library,Library,7.5,15.0,"2161 Queen St E (at Lee St.), Toronto ON M4L 1J1",(416) 393-7703
183,Toronto Public Library,Library,7.5,12.0,"40 St Clair Ave E (at Yonge St.), Toronto ON M...",(416) 393-7657


YELP

Top 10 POI- Vegetarian restaurants by rating

In [230]:
df_yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3677 entries, 0 to 3676
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          3677 non-null   object 
 1   closed        3677 non-null   bool   
 2   review        3677 non-null   int64  
 3   rating        3677 non-null   float64
 4   categories    3677 non-null   object 
 5   phone         3677 non-null   object 
 6   distance      3677 non-null   float64
 7   latitude      3677 non-null   float64
 8   longitude     3677 non-null   float64
 9   address       3677 non-null   object 
 10  station_name  3677 non-null   object 
dtypes: bool(1), float64(4), int64(1), object(5)
memory usage: 291.0+ KB


YELP- Remove duplicates and consider POI NAME, RATING, REVIEW, ADDRESS, TELEPHONE TO DISPLAY TOP 10 RATED RESTAURANT

In [231]:

columns_to_copy=['name','categories','rating','review','address','phone']
df_yelp_selectedcolumn=df_yelp[columns_to_copy].copy()
df_yelp_selectedcolumn=df_yelp_selectedcolumn.drop_duplicates()


In [239]:
yelp_top_poi=df_yelp_selectedcolumn.sort_values(by='rating',ascending=False)
yelp_top_resturants=yelp_top_poi[(yelp_top_poi['categories']!='Libraries') & (yelp_top_poi['categories']!='Museums')]
yelp_top_resturants.head(20)


,name,categories,rating,review,address,phone
277,Sunny Cafe,Vegetarian,5.0,3,"40 Saint George Street, Room 6141, Toronto, ON...",
1908,KaSpace Cafe,Vegetarian,5.0,9,"765 Queen Street E, Toronto, ON M4M 1H3, Canada",+16476579600
3350,KaSpace Cafe,Vegetarian,5.0,9,"1326 Gerrard Street E, Toronto, ON M4L 1Z1, C...",+16476579600
1931,Sunny Cafe,Vegetarian,5.0,3,"789 Yonge Street, Toronto, ON M4W 2G8, Canada",
1,KaSpace Cafe,Vegetarian,5.0,9,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",+16476579600
127,Sunny Cafe,Vegetarian,5.0,3,"2 Bloor Street W, Toronto, ON M4W 3E2, Canada",
98,Sunny Cafe,Vegetarian,5.0,3,"10 Spadina Road, Toronto, ON M5R 2S7, Canada",
198,KaSpace Cafe,Vegetarian,5.0,9,"Jones 118 Jones Ave, Toronto, ON M4M 2Z9, Canada",+16476579600
3031,Sunny Cafe,Vegetarian,5.0,3,"55 Avenue Rd, Toronto, ON M5R 3L2, Canada",
971,Sunny Cafe,Vegetarian,5.0,3,"322 Bloor Street W, Toronto, ON M5S 1W5, Canada",


In [240]:
yelp_top_libraries=yelp_top_poi[(yelp_top_poi['categories']=='Libraries') & (yelp_top_poi['categories']!='Museums')]
yelp_top_libraries.head(20)


,name,categories,rating,review,address,phone
1928,John W. Graham Library,Libraries,5.0,2,"789 Yonge Street, Toronto, ON M4W 2G8, Canada",+14169785851
375,John M Kelly Library,Libraries,5.0,1,"113 St. Joseph Street, Toronto, ON M5S 3C2, Ca...",
276,John W. Graham Library,Libraries,5.0,2,"40 Saint George Street, Room 6141, Toronto, ON...",+14169785851
1889,John W. Graham Library,Libraries,5.0,2,"Toronto Reference Library, 789 Yonge St, 2nd f...",+14169785851
65,John W. Graham Library,Libraries,5.0,2,"10 King's College Road, Room 2402, Toronto, ON...",+14169785851
1243,John M Kelly Library,Libraries,5.0,1,"341 Yonge Street, Toronto, ON M5B 1S1, Canada",
1249,John W. Graham Library,Libraries,5.0,2,"341 Yonge Street, Toronto, ON M5B 1S1, Canada",+14169785851
1700,John W. Graham Library,Libraries,5.0,2,"118-92 King Street E, Toronto, ON M5C 2V8, Canada",+14169785851
145,John W. Graham Library,Libraries,5.0,2,"15 King's College Cir, 2nd Fl, N Wing, Toronto...",+14169785851
3119,John W. Graham Library,Libraries,5.0,2,"56 Wellesley St W, Toronto, ON M5S 2S3, Canada",+14169785851


In [203]:
print(df_yelp.dtypes)

name             object
closed             bool
review            int64
rating          float64
categories       object
phone            object
distance        float64
latitude        float64
longitude       float64
address          object
station_name     object
dtype: object


In [224]:
foursquare_top_poi=df_four.sort_values(by='rating',ascending=False)
foursquare_top_resturants=foursquare_top_poi[foursquare_top_poi['category']!='Library'].head(10)
foursquare_top_resturants

,name,distance,category,latitude,longitude,rating,reviews,description,formatted_address,telephone,tips,hours,station_name
1338,Fresh Kitchen + Juice Bar Front,497,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Frederick St / The Esplanade
1380,Fresh Kitchen + Juice Bar Front,688,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Queens Quay / Yonge St
1349,Fresh Kitchen + Juice Bar Front,498,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,King St W / Bay St (West Side)
210,Fresh Kitchen + Juice Bar Front,701,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,King St W / York St
1390,Fresh Kitchen + Juice Bar Front,402,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Front St W / Bay St (North Side)
2980,Fresh Kitchen + Juice Bar Front,535,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Richmond St E / Jarvis St Green P
2940,Fresh Kitchen + Juice Bar Front,908,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Mutual St / Shuter St (Arena Gardens)
2885,Fresh Kitchen + Juice Bar Front,661,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Bay St / Harbour St (East)
1328,Fresh Kitchen + Juice Bar Front,85,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,Front St E / Scott St
3591,Fresh Kitchen + Juice Bar Front,898,Vegan and Vegetarian Restaurant,43.647823,-79.374336,8.9,19.0,None,"47 Front St E (Church Street), Toronto ON M5E 1B3",(647) 693-7556,Buffalo cauliflower. Got tons of zing!,Mon-Fri 11:00-22:00; Sat-Sun 10:30-22:00,York St / Harbour St (Love Park)


In [205]:
yelp_top_poi=df_yelp.sort_values(by='rating',ascending=False)
yelp_top_resturants=yelp_top_poi[(yelp_top_poi['categories']!='Library') & (yelp_top_poi['categories']!='Museums')]
yelp_top_resturants.head(10)

,name,closed,review,rating,categories,phone,distance,latitude,longitude,address,station_name
2438,John W. Graham Library,False,2,5.0,Libraries,+14169785851,828.899614,43.66501,-79.39660,"10 King's College Road, Room 2402, Toronto, ON...",Dalton Rd / Bloor St W
198,KaSpace Cafe,False,9,5.0,Vegetarian,+16476579600,835.200890,43.66263,-79.33190,"Jones 118 Jones Ave, Toronto, ON M4M 2Z9, Canada",Morse St / Eastern Ave
1309,Sunny Cafe,False,3,5.0,Vegetarian,,673.595590,43.66694,-79.40321,"10 Spadina Road, Toronto, ON M5R 2S7, Canada",Bathurst Subway Station
1,KaSpace Cafe,False,9,5.0,Vegetarian,+16476579600,1023.018862,43.66263,-79.33190,"1432 Gerrard Street East, Toronto, ON M4L 1Z6,...",Queen St E / Woodward Ave
753,KaSpace Cafe,False,9,5.0,Vegetarian,+16476579600,454.047204,43.66263,-79.33190,"Jones 118 Jones Ave, Toronto, ON M4M 2Z9, Canada",Lake Shore Blvd E / Leslie St
3522,John W. Graham Library,False,2,5.0,Libraries,+14169785851,520.594666,43.66501,-79.39660,"40 Saint George Street, Room 6141, Toronto, ON...",Spadina Ave / Sussex Ave - SMART
3521,Sunny Cafe,False,3,5.0,Vegetarian,,325.968568,43.66694,-79.40321,"40 Saint George Street, Room 6141, Toronto, ON...",Spadina Ave / Sussex Ave - SMART
741,Sunny Cafe,False,3,5.0,Vegetarian,,648.193619,43.66694,-79.40321,"15 King's College Cir, 2nd Fl, N Wing, Toronto...",Kendal Ave / Bernard Ave
737,John W. Graham Library,False,2,5.0,Libraries,+14169785851,1210.333758,43.66501,-79.39660,"15 King's College Cir, 2nd Fl, N Wing, Toronto...",Kendal Ave / Bernard Ave
726,Sunny Cafe,False,3,5.0,Vegetarian,,1240.193859,43.66694,-79.40321,"10 Spadina Road, Toronto, ON M5R 2S7, Canada",Bridgman Ave / Bathurst St


In [206]:
unique_categories = df_four['category'].unique()
print(unique_categories)

['Library' 'Coffee Shop' 'Deli' 'Vegan and Vegetarian Restaurant'
 'Mexican' 'Fast Food' 'Salad' 'Burgers' 'Ice Cream' 'Caribbean'
 'Hair Salon' 'Falafel' 'Diner' 'Indian' 'Café' 'Bakery' 'Sushi' 'Buffet'
 'Hawaiian' 'Publisher' 'Thai' 'Spa' 'Lounge' 'Arts & Entertainment'
 'Chinese' 'Cocktail' 'Soup Spot' 'South Indian' 'Sandwich Spot'
 'Art Museum' 'Ethiopian' 'Architecture Firm' 'Juice Bar' 'Restaurant'
 'Gluten-free']


In [207]:
unique_categories = df_yelp['categories'].unique()
print(unique_categories)

['Indian' 'Vegetarian' 'Vegan' 'Libraries' 'Tapas/Small Plates' 'Salad'
 'Gluten-Free' 'Breakfast & Brunch' 'Museums' 'Vietnamese' 'Falafel'
 'Middle Eastern' 'Thai' 'Fast Food' 'Delis']


In [208]:
df_yelp_resturants=df_yelp[df_yelp['categories']=='Libraries']
df_yelp_resturants.tail(10)

,name,closed,review,rating,categories,phone,distance,latitude,longitude,address,station_name
3643,Fort York Library,False,19,3.5,Libraries,+14163936240,802.872055,43.639007,-79.400075,"1171 Dundas Street W, Toronto, ON M6J 1X3, Canada",Wellington St W / Stafford St
3647,Queen/Saulter Public Library,False,6,3.5,Libraries,+14163937723,195.800619,43.659190,-79.347540,"370 Broadview Avenue, Toronto, ON M4M 2H1, Canada",111 Broadview Ave
3648,Toronto Public Library - Riverdale,False,4,4.0,Libraries,+14163937720,762.709499,43.665810,-79.353140,"370 Broadview Avenue, Toronto, ON M4M 2H1, Canada",111 Broadview Ave
3652,Toronto Public Library - Palmerston Branch,False,5,4.0,Libraries,+14163937680,924.996991,43.665090,-79.414150,"1246 Shaw Street, Toronto, ON M6G 3N9, Canada",Christie St / Dupont St
3653,Toronto Public Library,False,3,2.5,Libraries,+14163937732,664.365857,43.674570,-79.428380,"1246 Shaw Street, Toronto, ON M6G 3N9, Canada",Christie St / Dupont St
3666,Ryerson University Student Learning Centre,False,4,3.5,Libraries,+14169795000,788.934959,43.657670,-79.381100,"100 Queen Street W, Nathan Phillips Square, To...",Richmond St E / Jarvis St Green P
3673,City Hall Public Library,False,6,3.5,Libraries,+14163937650,776.740866,43.653347,-79.383826,"100 Queen Street W, Nathan Phillips Square, To...",Richmond St E / Jarvis St Green P
3674,Toronto Public Library - Parliament Street Branch,False,6,2.0,Libraries,+14163937663,314.733258,43.661830,-79.367910,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St
3675,Toronto Public Library - Riverdale,False,4,4.0,Libraries,+14163937720,1010.279295,43.665810,-79.353140,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St
3676,Toronto Public Library - St. James Town Branch,False,3,4.0,Libraries,+14163937744,1068.656684,43.667890,-79.374690,"495 Sherbourne St, Toronto, ON M4X 1L1, Canada",Oak St / Sackville St


In [209]:
df_four_lib=df_four[df_four['category']=='Library']
df_four_lib.tail(10)

,name,distance,category,latitude,longitude,rating,reviews,description,formatted_address,telephone,tips,hours,station_name
4183,Toronto Public Library,917,Library,43.674262,-79.460723,NaN,NaN,None,"1748 St Clair Ave W, Toronto ON M6N 1J3",(416) 393-7709,None,Tue 12:30-20:30; Wed 10:00-18:00; Thu 12:30-20...,Gunns Rd / Lavender Creek Trail
4184,Boneshaker Reading Series,917,Library,43.674262,-79.460723,NaN,NaN,None,"1748 St Clair Ave W, Toronto ON",(416) 393-7710,None,None,Gunns Rd / Lavender Creek Trail
4185,Toronto Public Library - Toronto Reference Lib...,319,Library,43.671847,-79.386762,9.0,378.0,Main intersection: Bloor and Yonge,"789 Yonge St (btwn Collier & Asquith Ave), Tor...",(416) 395-5577,This is Toronto Public Library’s main location...,Mon-Fri 9:00-20:30; Sat 9:00-17:00; Sun 13:30-...,20 Charles St E
4190,Toronto Public Library,845,Library,43.667774,-79.374796,6.5,12.0,None,"495 Sherbourne St (Wellesley Ave), Toronto ON ...",(416) 393-7744,very limited number of free museum tickets. ot...,Tue 12:30-20:30; Wed 10:00-18:00; Thu 12:30-20...,20 Charles St E
4192,Canadian Lesbian and Gay Archives,178,Library,43.668057,-79.384054,NaN,NaN,None,"34 Isabella St (at Yonge St.), Toronto ON M4Y 1N1",(416) 777-2755,Did you know that the CLGA was founded in 1973...,Tue-Thu 19:30-22:00; Fri 13:00-17:00,20 Charles St E
4195,Brentwood Public Library,914,Library,43.647480,-79.514244,7.4,9.0,None,"36 Brentwood Rd N (at Bloor St. W), Toronto ON...",(416) 394-5240,Second floor is a circus of noise performed by...,Mon-Fri 9:00-20:30; Sat 9:00-17:00; Sun 13:30-...,Bloor St W / Kingsmill Rd
4202,Toronto Public Library - Palmerston Branch,203,Library,43.665201,-79.414090,6.1,8.0,None,"560 Palmerston Ave (at Bloor St. W), Toronto O...",(416) 393-7680,Not enough computers.,Mon 10:00-20:30; Tue 12:30-20:30; Wed 10:00-18...,Palmerston Blvd / Lennox St
4203,Toronto Public Library,186,Library,43.664934,-79.413907,NaN,NaN,None,"2560 Palmerston Ave, Toronto ON M6G 2P7",(416) 393-7680,None,Mon 10:00-20:30; Tue 12:30-20:30; Wed 10:00-18...,Palmerston Blvd / Lennox St
4212,Toronto Public Library - Palmerston Branch,341,Library,43.665201,-79.414090,6.1,8.0,None,"560 Palmerston Ave (at Bloor St. W), Toronto O...",(416) 393-7680,Not enough computers.,Mon 10:00-20:30; Tue 12:30-20:30; Wed 10:00-18...,Palmerston Blvd / Herrick St
4213,Toronto Public Library,324,Library,43.664934,-79.413907,NaN,NaN,None,"2560 Palmerston Ave, Toronto ON M6G 2P7",(416) 393-7680,None,Mon 10:00-20:30; Tue 12:30-20:30; Wed 10:00-18...,Palmerston Blvd / Herrick St


YELP - DISPLAY UNIQUE POI NAME, ADDRESS,CATEGORIES 

YELP Library

In [214]:
distinct_data = df_yelp[['name', 'address','categories']].drop_duplicates()


cnt_lib=distinct_data[distinct_data['categories']=='Libraries'].shape[0]
cnt_vegrest=distinct_data[distinct_data['categories']!='Libraries'].shape[0]
print(cnt_lib)
print(cnt_vegrest)
# # # Convert tuples back to lists if needed
# distinct_data['address'] = distinct_data['address'].apply(list)

# # # Display the distinct_data DataFrame
# # print(distinct_data)


463
723


YELP - VEGETARUAN RESTAURANTS 

In [215]:
# Display the distinct values

# Count of unique library names
library_counts = distinct_data[distinct_data['categories'] == 'Libraries']['name'].value_counts()
vegresturant_counts = distinct_data[distinct_data['categories'] != 'Libraries']['name'].value_counts()

print("\nUnique library names count:")
print(library_counts)
print("\nUnique vegetarian resturants names count:")
print(vegresturant_counts)



Unique library names count:
name
Toronto Public Library                                            48
John W. Graham Library                                            23
Ryerson University Student Learning Centre                        23
Gerstein Science Information Centre                               22
Robarts Library                                                   20
Thomas Fisher Rare Book Library                                   19
City Hall Public Library                                          19
EJ Pratt Libary                                                   19
Fort York Library                                                 18
Yorkville Library                                                 14
The Japan Foundation, Toronto                                     14
OISE Library                                                      14
Toronto Reference Library                                         13
Toronto Public Library - Parliament Street Branch                 13


FOURSQUARE - DISPLAY UNIQUE POI NAME, ADDRESS,CATEGORIES 

In [216]:
distinct_data = df_four[['name', 'formatted_address','category']].drop_duplicates()
library_cnt=distinct_data[distinct_data['category']=='Library'].shape[0]
print(library_cnt)
vegrest_cnt=distinct_data[distinct_data['category']!='Library'].shape[0]
print(vegrest_cnt)

140
135


In [217]:
# Display the distinct values
#Count of unique restaurant names
restaurant_counts = distinct_data[distinct_data['category'] != 'Library']['name'].value_counts()

#Count of unique rlibrary names
library_counts = distinct_data[distinct_data['category'] == 'Library']['name'].value_counts()


print("Unique restaurant names count:")
print(restaurant_counts)
print("Unique library names count:")
print(library_counts)

Unique restaurant names count:
name
Freshii                                 17
Kupfert & Kim                            6
Calii Love                               2
Copper Branch                            2
Aesop                                    2
                                        ..
Swiss Chalet Rotisserie & Grill          1
Pho Some Restaurant                      1
Live Food Bar                            1
Ampersand Inc                            1
Flower Lantern Vegetarian Restaurant     1
Name: count, Length: 109, dtype: int64
Unique library names count:
name
Toronto Public Library                41
North York Public Library              4
Corporation of the City of Toronto     3
York Public Library                    3
D Films Corp                           2
                                      ..
Boneshaker Reading Series              1
Function in the Junction               1
Minh Phung                             1
Annette Branch Public Library          1
Woodview 




Foursquare Vegetarina resturants - 140
Foursquare library - 135


Top 10 Vegetarian Friendly Restaurants 
Top 10 Libraries



<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>category</th>
      <th>rating</th>
      <th>reviews</th>
      <th>formatted_address</th>
      <th>telephone</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>9</th>
      <td>Fresh Kitchen + Juice Bar Front</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.9</td>
      <td>19.0</td>
      <td>47 Front St E (Church Street), Toronto ON M5E 1B3</td>
      <td>(647) 693-7556</td>
    </tr>
    <tr>
      <th>50</th>
      <td>Fresh on Bloor</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.8</td>
      <td>380.0</td>
      <td>326 Bloor St W, Toronto ON M5S 1W5</td>
      <td>(416) 599-4442</td>
    </tr>
    <tr>
      <th>442</th>
      <td>Tori's Bakeshop</td>
      <td>Bakery</td>
      <td>8.7</td>
      <td>78.0</td>
      <td>2188 Queen St E (Balsam Ave), Toronto ON M4E 1E6</td>
      <td>(647) 350-6500</td>
    </tr>
    <tr>
      <th>10</th>
      <td>Rosalinda</td>
      <td>Mexican</td>
      <td>8.7</td>
      <td>35.0</td>
      <td>133 Richmond St W, Toronto ON M5H 2L3</td>
      <td>(416) 907-0650</td>
    </tr>
    <tr>
      <th>49</th>
      <td>One Love Vegetarian</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.4</td>
      <td>51.0</td>
      <td>854 Bathurst St (at London St), Toronto ON M5R...</td>
      <td>(416) 535-5683</td>
    </tr>
    <tr>
      <th>478</th>
      <td>Hello 123</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.4</td>
      <td>36.0</td>
      <td>1122 Queen St W, Toronto ON M6J 1H9</td>
      <td>(416) 532-3555</td>
    </tr>
    <tr>
      <th>345</th>
      <td>YamChops</td>
      <td>Soup Spot</td>
      <td>8.4</td>
      <td>29.0</td>
      <td>705 College St, Toronto ON M6G 1C2</td>
      <td>(416) 645-0117</td>
    </tr>
    <tr>
      <th>403</th>
      <td>Fresh on Crawford</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.3</td>
      <td>251.0</td>
      <td>894 Queen St W (at Crawford), Toronto ON M6J 1G3</td>
      <td>(416) 599-4442</td>
    </tr>
    <tr>
      <th>255</th>
      <td>Buddha's Vegetarian Foods</td>
      <td>Chinese</td>
      <td>8.3</td>
      <td>48.0</td>
      <td>666 Dundas St W (Bathurst St.), Toronto ON M5T...</td>
      <td>(416) 603-3811</td>
    </tr>
    <tr>
      <th>190</th>
      <td>Fresh on Spadina</td>
      <td>Vegan and Vegetarian Restaurant</td>
      <td>8.2</td>
      <td>419.0</td>
      <td>147 Spadina Ave (at Richmond St. W.), Toronto ...</td>
      <td>(416) 599-4442</td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>category</th>
      <th>rating</th>
      <th>reviews</th>
      <th>formatted_address</th>
      <th>telephone</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>59</th>
      <td>Toronto Public Library - Toronto Reference Lib...</td>
      <td>Library</td>
      <td>9.0</td>
      <td>378.0</td>
      <td>789 Yonge St (btwn Collier &amp; Asquith Ave), Tor...</td>
      <td>(416) 395-5577</td>
    </tr>
    <tr>
      <th>189</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>8.6</td>
      <td>42.0</td>
      <td>190 Fort York Blvd, Toronto ON M5V 0E7</td>
      <td>(416) 393-6240</td>
    </tr>
    <tr>
      <th>30</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>8.3</td>
      <td>38.0</td>
      <td>239 College St (at Huron St.), Toronto ON M5T 1R5</td>
      <td>(416) 393-7746</td>
    </tr>
    <tr>
      <th>89</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>8.0</td>
      <td>11.0</td>
      <td>370 Broadview Ave (at Gerrard St E), Toronto O...</td>
      <td>(416) 393-7720</td>
    </tr>
    <tr>
      <th>325</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.9</td>
      <td>51.0</td>
      <td>1101 Bloor St W (at Gladstone), Toronto ON M6H...</td>
      <td>(416) 393-7674</td>
    </tr>
    <tr>
      <th>1597</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.8</td>
      <td>15.0</td>
      <td>3083 Yonge St (at Lawrence Ave.), Toronto ON M...</td>
      <td>(416) 393-7730</td>
    </tr>
    <tr>
      <th>146</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.6</td>
      <td>14.0</td>
      <td>701 Pape Ave (at Danforth Ave), Toronto ON M4K...</td>
      <td>(416) 393-7727</td>
    </tr>
    <tr>
      <th>3687</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.6</td>
      <td>8.0</td>
      <td>496 Birchmount Rd, Toronto ON M1K 1N8</td>
      <td>(416) 396-8890</td>
    </tr>
    <tr>
      <th>443</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.5</td>
      <td>15.0</td>
      <td>2161 Queen St E (at Lee St.), Toronto ON M4L 1J1</td>
      <td>(416) 393-7703</td>
    </tr>
    <tr>
      <th>183</th>
      <td>Toronto Public Library</td>
      <td>Library</td>
      <td>7.5</td>
      <td>12.0</td>
      <td>40 St Clair Ave E (at Yonge St.), Toronto ON M...</td>
      <td>(416) 393-7657</td>
    </tr>
  </tbody>
</table>
</div>


Yelp Vegetarian resturants- 723 entries
Yelp library  - 463
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>categories</th>
      <th>rating</th>
      <th>review</th>
      <th>address</th>
      <th>phone</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>277</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>40 Saint George Street, Room 6141, Toronto, ON...</td>
      <td></td>
    </tr>
    <tr>
      <th>1908</th>
      <td>KaSpace Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>9</td>
      <td>765 Queen Street E, Toronto, ON M4M 1H3, Canada</td>
      <td>+16476579600</td>
    </tr>
    <tr>
      <th>3350</th>
      <td>KaSpace Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>9</td>
      <td>1326 Gerrard Street  E, Toronto, ON M4L 1Z1, C...</td>
      <td>+16476579600</td>
    </tr>
    <tr>
      <th>1931</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>789 Yonge Street, Toronto, ON M4W 2G8, Canada</td>
      <td></td>
    </tr>
    <tr>
      <th>1</th>
      <td>KaSpace Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>9</td>
      <td>1432 Gerrard Street East, Toronto, ON M4L 1Z6,...</td>
      <td>+16476579600</td>
    </tr>
    <tr>
      <th>127</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>2 Bloor Street W, Toronto, ON M4W 3E2, Canada</td>
      <td></td>
    </tr>
    <tr>
      <th>98</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>10 Spadina Road, Toronto, ON M5R 2S7, Canada</td>
      <td></td>
    </tr>
    <tr>
      <th>198</th>
      <td>KaSpace Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>9</td>
      <td>Jones 118 Jones Ave, Toronto, ON M4M 2Z9, Canada</td>
      <td>+16476579600</td>
    </tr>
    <tr>
      <th>3031</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>55 Avenue Rd, Toronto, ON M5R 3L2, Canada</td>
      <td></td>
    </tr>
    <tr>
      <th>971</th>
      <td>Sunny Cafe</td>
      <td>Vegetarian</td>
      <td>5.0</td>
      <td>3</td>
      <td>322 Bloor Street W, Toronto, ON M5S 1W5, Canada</td>
      <td></td>
    </tr>
  </tbody>
</table>
</div>

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>categories</th>
      <th>rating</th>
      <th>review</th>
      <th>address</th>
      <th>phone</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1928</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>789 Yonge Street, Toronto, ON M4W 2G8, Canada</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>375</th>
      <td>John M Kelly Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>1</td>
      <td>113 St. Joseph Street, Toronto, ON M5S 3C2, Ca...</td>
      <td></td>
    </tr>
    <tr>
      <th>276</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>40 Saint George Street, Room 6141, Toronto, ON...</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>1889</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>Toronto Reference Library, 789 Yonge St, 2nd f...</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>65</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>10 King's College Road, Room 2402, Toronto, ON...</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>1243</th>
      <td>John M Kelly Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>1</td>
      <td>341 Yonge Street, Toronto, ON M5B 1S1, Canada</td>
      <td></td>
    </tr>
    <tr>
      <th>1249</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>341 Yonge Street, Toronto, ON M5B 1S1, Canada</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>1700</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>118-92 King Street E, Toronto, ON M5C 2V8, Canada</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>145</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>15 King's College Cir, 2nd Fl, N Wing, Toronto...</td>
      <td>+14169785851</td>
    </tr>
    <tr>
      <th>3119</th>
      <td>John W. Graham Library</td>
      <td>Libraries</td>
      <td>5.0</td>
      <td>2</td>
      <td>56 Wellesley St W, Toronto, ON M5S 2S3, Canada</td>
      <td>+14169785851</td>
    </tr>
  </tbody>
</table>
</div>



Yelp is giving more coverage for my location. hence going ahead with yelp.